In [ ]:
max_match_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_match_ts", debugValue="2026-01-01 00:00:00")
max_delivery_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_delivery_ts", debugValue="2026-01-01 00:00:00")

print(max_match_ts, max_delivery_ts)

In [ ]:
delivery_df = spark.sql(f"SELECT * FROM bronze.delivery_loading WHERE ingestion_ts <= '{max_delivery_ts}'")

dim_team = spark.read.table("silver.dim_team") \
    .select("team_sk", "team_name")

dim_player = spark.read.table("silver.dim_player") \
    .select("player_sk", "player_name")


In [ ]:
df = (
    delivery_df
    .join(
        dim_team.withColumnRenamed("team_sk", "batting_team_sk")
                .withColumnRenamed("team_name", "batting_team"),
        "batting_team",
        "left"
    )
    .join(
        dim_team.withColumnRenamed("team_sk", "bowling_team_sk")
                .withColumnRenamed("team_name", "bowling_team"),
        "bowling_team",
        "left"
    )
)


In [ ]:
df = (
    df
    .join(
        dim_player.withColumnRenamed("player_sk", "batsman_sk")
                  .withColumnRenamed("player_name", "batsman"),
        "batsman",
        "left"
    )
    .join(
        dim_player.withColumnRenamed("player_sk", "non_striker_sk")
                  .withColumnRenamed("player_name", "non_striker"),
        "non_striker",
        "left"
    )
    .join(
        dim_player.withColumnRenamed("player_sk", "bowler_sk")
                  .withColumnRenamed("player_name", "bowler"),
        "bowler",
        "left"
    )
    .join(
        dim_player.withColumnRenamed("player_sk", "player_dismissed_sk")
                  .withColumnRenamed("player_name", "player_dismissed"),
        "player_dismissed",
        "left"
    )
    .join(
        dim_player.withColumnRenamed("player_sk", "fielder_sk")
                  .withColumnRenamed("player_name", "fielder"),
        "fielder",
        "left"
    )
)


In [ ]:
from pyspark.sql.functions import current_timestamp
fact_df = df.select(
    "match_id",
    "inning",
    "over",
    "ball",
    "is_super_over",

    "batting_team_sk",
    "bowling_team_sk",

    "batsman_sk",
    "non_striker_sk",
    "bowler_sk",
    "player_dismissed_sk",
    "fielder_sk",

    "wide_runs",
    "bye_runs",
    "legbye_runs",
    "noball_runs",
    "penalty_runs",
    "batsman_runs",
    "extra_runs",
    "total_runs",

    "dismissal_kind",
    "source_file",
    "ingestion_ts",

    current_timestamp().alias("created_at")
)


In [ ]:
(
    fact_df
    .write
    .format("delta")
    .mode("append")
    .saveAsTable("silver.fact_deliveries")
)
